In [1]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install pywhisper

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.3/388.3 KB 40.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 81.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 26.9/26.9 MB 58.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 23.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 87.5 MB/s eta 0:00:00
  Created wheel for moviepy: filename=moviepy-1.0.3-py3-none-any.whl size=110742 sha256=38d101bce7c518018f24effda78498bb349393e10048d406c51f277fbf5d721e
  Stored in directory: /root/.cache/pip/wheels/e4/a4/db/0368d3a04033da662e13926594b3a8cf1aa4ffeefe570cfac1
Successfully built moviepy
  Attempting uninstall: moviepy
    Found existing installation: moviepy 0.2.3.5
    Uninstall

In [3]:
!pip install jiwer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 72.0 MB/s eta 0:00:00


In [4]:
import pywhisper
from pywhisper.utils import write_srt, format_timestamp
from pathlib import Path

model_Large = pywhisper.load_model("large")

100%|█████████████████████████████████████| 2.87G/2.87G [00:52<00:00, 58.5MiB/s]


In [5]:
import csv
import os
from jiwer import wer

In [6]:
import re


In [7]:
working_directory = '/content/drive/MyDrive/Apziva/fake_audio_detection'

In [8]:
fake_voices_dir = os.path.join(working_directory, 'fake_speeches')

In [9]:
fake_list = os.listdir(fake_voices_dir)

In [10]:
fake_list[2]

'fake_1905.wav'

In [11]:
not_found_files = []
for i in range(2001):
  if 'fake_{}.wav'.format(i) not in fake_list:
    #print(i)
    not_found_files.append(i)

In [12]:
len(not_found_files)

33

In [13]:
test_result_dict = model_Large.transcribe(os.path.join(fake_voices_dir,fake_list[2]))

In [14]:
test_result_dict['text']

' The publication of this report raised a storm in the city, and the Corporation was roused to make an immediate protest.'

In [15]:
clean_string = [s for s in test_result_dict['text'] if s.isalnum() or s.isspace()]
clean_string = "".join(clean_string)

In [16]:
clean_string

' The publication of this report raised a storm in the city and the Corporation was roused to make an immediate protest'

In [17]:
re.sub('\W+ ',' ', test_result_dict['text']).lower()

' the publication of this report raised a storm in the city and the corporation was roused to make an immediate protest.'

In [18]:
error_rate = wer(clean_string, clean_string)

In [19]:
print(error_rate)

0.0


In [ ]:
error_list = []
path = fake_voices_dir

with open(os.path.join(working_directory, 'metadata.csv')) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='|')
    line_count = 0
    for row in csv_reader:
        if line_count in not_found_files:
          line_count+=1
          continue
        try:
            
            text_from_metadata=row[2]
            filename = fake_list[line_count]
            model_output = model_Large.transcribe(os.path.join(path,filename))

            model_output_text = model_output.get('text')
            #print(model_output_text)
      
            clean_string = [s for s in model_output_text if s.isalnum() or s.isspace()]
            clean_string = "".join(clean_string)
            clean_string = clean_string.lower()

            text_from_metadata = text_from_metadata.lower()
            text_from_metadata = [s for s in text_from_metadata if s.isalnum() or s.isspace()]
            text_from_metadata = "".join(text_from_metadata)
            error_rate = wer(text_from_metadata, clean_string)
            #tts.tts_to_file(text=text_from_metadata, file_path='fake_speeches/fake_{}.wav'.format(line_count))
            error_list.append(error_rate)
        except Exception as e:
            print('problem processing line: {}'.format(line_count))
            print(e)
        line_count+=1
        if line_count == 2000:
            break
        
    print(f'Processed {line_count} lines.')

problem processing line: 1968
list index out of range
problem processing line: 1969
list index out of range
problem processing line: 1970
list index out of range
problem processing line: 1971
list index out of range
problem processing line: 1972
list index out of range
problem processing line: 1973
list index out of range
problem processing line: 1974
list index out of range
problem processing line: 1975
list index out of range
problem processing line: 1976
list index out of range
problem processing line: 1977
list index out of range
problem processing line: 1978
list index out of range
problem processing line: 1979
list index out of range
problem processing line: 1980
list index out of range
problem processing line: 1981
list index out of range
problem processing line: 1982
list index out of range
problem processing line: 1983
list index out of range
problem processing line: 1984
list index out of range
problem processing line: 1985
list index out of range
problem processing line: 198

In [ ]:
import pickle

In [ ]:
with open('error_list_pickled.pkl', 'wb') as f:
  pickle.dump(error_list, f)

In [ ]:
def Average(lst):
    return sum(lst) / len(lst)

In [ ]:
average = Average(error_list)

In [ ]:
print(average)